In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [9]:
df = pd.read_csv("songs.csv")


In [10]:
singer_encoder = LabelEncoder()
track_encoder = LabelEncoder()

In [11]:
df['SINGER_NAME_ENCODED'] = singer_encoder.fit_transform(df['SINGER_NAME'])
df['TRACK_NAME_ENCODED'] = track_encoder.fit_transform(df['TRACK_NAME'])



In [12]:
X = df['SINGER_NAME_ENCODED'].values.reshape(-1, 1)
y = df['TRACK_NAME_ENCODED'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
model = Sequential([
    Embedding(input_dim=len(singer_encoder.classes_), output_dim=50, input_length=1),
    LSTM(100),
    Dense(len(track_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [75]:
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))


Epoch 1/50
4/4 [==============================] - 0s 42ms/step - loss: 0.2468 - accuracy: 0.8326 - val_loss: 10.0782 - val_accuracy: 0.0000e+00
Epoch 2/50
4/4 [==============================] - 0s 19ms/step - loss: 0.2464 - accuracy: 0.8455 - val_loss: 10.0794 - val_accuracy: 0.0000e+00
Epoch 3/50
4/4 [==============================] - 0s 21ms/step - loss: 0.2466 - accuracy: 0.8283 - val_loss: 10.0809 - val_accuracy: 0.0000e+00
Epoch 4/50
4/4 [==============================] - 0s 20ms/step - loss: 0.2466 - accuracy: 0.8412 - val_loss: 10.0811 - val_accuracy: 0.0000e+00
Epoch 5/50
4/4 [==============================] - 0s 23ms/step - loss: 0.2465 - accuracy: 0.8283 - val_loss: 10.0824 - val_accuracy: 0.0000e+00
Epoch 6/50
4/4 [==============================] - 0s 19ms/step - loss: 0.2465 - accuracy: 0.8369 - val_loss: 10.0848 - val_accuracy: 0.0000e+00
Epoch 7/50
4/4 [==============================] - 0s 17ms/step - loss: 0.2468 - accuracy: 0.8326 - val_loss: 10.0873 - val_accuracy: 0.0

In [63]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

2/2 [==============================] - 0s 11ms/step - loss: 9.7102 - accuracy: 0.0000e+00
Test Accuracy: 0.0


In [76]:
def generate_songs(singer_name, num_songs=5):
    singer_encoded = singer_encoder.transform([singer_name])
    singer_encoded = np.array(singer_encoded).reshape(-1, 1)
    generated_songs = []
    for _ in range(num_songs):
        prediction = model.predict(singer_encoded)
        sampled_index = np.random.choice(len(prediction[0]), p=prediction[0])
        sampled_track = track_encoder.inverse_transform([sampled_index])[0]
        generated_songs.append(sampled_track)
    return generated_songs


In [92]:
singer_name = "Doja Cat"
num_songs = 3
generated_songs = generate_songs(singer_name, num_songs=num_songs)
print(f"Generated songs for singer {singer_name}:")
for i, song in enumerate(generated_songs, 1):
    print(f"Song {i}: {song}")

1/1 [==============================] - 0s 29ms/step
Generated songs for singer Doja Cat:
Song 1: Say So
Song 2: Like That (feat. Gucci Mane)
Song 3: Freak
